In [ ]:
import os
import pymysql

from dotenv import load_dotenv

load_dotenv()

def get_conn():
    connection_params = {
        "host": os.environ["MYSQL_HOST"],
        "port": int(os.environ["MYSQL_PORT"]),
        "user": os.environ["MYSQL_USER"],
        "password": os.environ["MYSQL_PASSWORD"],
        "db": os.environ["MYSQL_DB"],
    }
    connection_params["cursorclass"] = pymysql.cursors.DictCursor

    return pymysql.connect(**connection_params)

with get_conn() as con, con.cursor() as cursor:
    cursor.execute(
        """
        SELECT
            ID
        FROM wPiM.pim_products
        WHERE wPiM.pim_products.product_status_sync2bw = 1;
        """
    )
    products_raw = cursor.fetchall()
products = [p['ID'] for p in products_raw]

with get_conn() as con, con.cursor() as cursor:
    cursor.execute(
        "SELECT attribute_productID, attribute_attribute_typeID FROM wPiM.pim_products_attributes;"
    )
    product_attributes_raw = cursor.fetchall()

In [6]:
print(f"Products:    {len(products)}")
print(f"Attributes: {len(product_attributes_raw)}")

Products:    14904
Attributes: 327513


In [ ]:
from collections import defaultdict

product_attributes = defaultdict(list)

for pa in product_attributes_raw:
    if pa['attribute_productID'] in products:
        product_attributes[pa['attribute_productID']].append(pa['attribute_attribute_typeID'])

print(f"Products with attributes: {len(product_attributes)}")